In [20]:
from kafka import KafkaConsumer
import json
try:
        # Create KafkaConsumer
        consumer = KafkaConsumer(
            "apmmetrics",
            bootstrap_servers=['telemetry-kafka-external-bootstrap-observability-kafka.apps.zagaopenshift.zagaopensource.com:443'],
            value_deserializer=lambda v: v,  # Keep value as bytes for JSON decoding
            group_id="apmingestion",
            auto_offset_reset='earliest',
            consumer_timeout_ms=10000,
            security_protocol='SSL',  # Add SSL configuration if needed
            ssl_cafile='./telemetry_zagaopenshift.pem',
            ssl_certfile='./telemetry_zagaopenshift.crt'
        )
        print("Kafka consumer created successfully.")
except Exception as e:
        print(f"Error creating Kafka consumer: {e}")

Kafka consumer created successfully.


In [21]:
messages=[]

try:
    for message in consumer:
        json_message=json.loads(message.value.decode('utf-8'))
        print(json_message)
        messages.append(json_message)
        
except Exception as e:
    print(f'error consuming message:{e}')

finally:
    consumer.close()
    print('message consuming completed')

print(messages)

    

{'resourceMetrics': [{'resource': {'attributes': [{'key': 'container.id', 'value': {'stringValue': 'a0bf9fa1ba205cac19ae9584e73de46d3c8de0cbe0be787f6a216eead1c0003b'}}, {'key': 'host.arch', 'value': {'stringValue': 'amd64'}}, {'key': 'host.name', 'value': {'stringValue': 'kafka-srv-5-59bfc54888-2gkzx'}}, {'key': 'k8s.container.name', 'value': {'stringValue': 'kafka-srv-5'}}, {'key': 'k8s.deployment.name', 'value': {'stringValue': 'kafka-srv-5'}}, {'key': 'k8s.namespace.name', 'value': {'stringValue': 'observability-testcases'}}, {'key': 'k8s.node.name', 'value': {'stringValue': 'zagaocp-worker2'}}, {'key': 'k8s.pod.name', 'value': {'stringValue': 'kafka-srv-5-59bfc54888-2gkzx'}}, {'key': 'k8s.replicaset.name', 'value': {'stringValue': 'kafka-srv-5-59bfc54888'}}, {'key': 'os.description', 'value': {'stringValue': 'Linux 5.14.0-284.71.1.el9_2.x86_64'}}, {'key': 'os.type', 'value': {'stringValue': 'linux'}}, {'key': 'process.command_args', 'value': {'arrayValue': {'values': [{'stringValue

In [22]:
from pyiceberg.catalog import load_catalog
catalog = load_catalog(
        "docs",
    **{
            "uri": "thrift://hive-ms-trino.apps.zagaopenshift.zagaopensource.com:9888",
            "s3.endpoint": "http://minio-lb.apps.zagaopenshift.zagaopensource.com:9009",
            "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
            "s3.access-key-id": "minioAdmin",
            "s3.secret-access-key": "minio1234",
    }
)

In [4]:
catalog.list_namespaces()

[('data',),
 ('default',),
 ('demo_data',),
 ('demo_saro',),
 ('demo_trace',),
 ('docs_check',),
 ('docs_logs',),
 ('docs_mari',),
 ('docs_powerleveldata',),
 ('docs_sa',),
 ('docs_sar',),
 ('docs_saro',),
 ('docs_selva',),
 ('hi',),
 ('hlo',),
 ('ingesting',),
 ('new',),
 ('old',),
 ('poc',),
 ('pod',),
 ('pod_data',),
 ('power_pod',),
 ('powerlevelpoddata',),
 ('powerpod',),
 ('saravan',),
 ('saravana',),
 ('saro',),
 ('saro_data',),
 ('sh',),
 ('sharanya',),
 ('sk',),
 ('soh',),
 ('trace_ingestion_data',)]

In [5]:
catalog.list_tables('docs_mari')

[('docs_mari', 'maari'),
 ('docs_mari', 'demo'),
 ('docs_mari', 'metrics'),
 ('docs_mari', 'metricsdata'),
 ('docs_mari', 'metric')]

In [23]:
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    LongType,
    IntegerType,
    StringType,
    NestedField,
    StructType,
    ListType
)
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import DayTransform
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.transforms import IdentityTransform

schema=Schema(
     NestedField(field_id=1, name="serviceName", field_type=StringType(), required=False),
     NestedField(field_id=2, name="createdTime", field_type=TimestampType(), required=False),
     NestedField(field_id=3, name="metricData", field_type=StringType(), required=False),
     NestedField(field_id=4, name="language", field_type=StringType(), required=False))

table=catalog.create_table(
    identifier="docs_mari.mymetrics3",
    schema=schema,
    location="s3a://mari/metricsData",)

In [24]:
from pyiceberg.io.pyarrow import PyArrowFileIO
import pyarrow as pa
import json
import datetime
metrics_datas=[]
all_data={}
for data in messages:
    for resourceMetric in data['resourceMetrics']:
        for attribute in resourceMetric['resource']['attributes']:
           #print(attribute)
           if attribute['key'] == 'service.name':
             
             all_data.update({
                "serviceName":attribute['value']['stringValue'],
                "createdTime":datetime.datetime.now(),
                 
             })

           if attribute['key'] =='telemetry.sdk.language':
              all_data.update({"metricData":json.dumps(data),"language":attribute['value']['stringValue']
                               })
              metrics_datas.append(all_data)
            
df=pa.Table.from_pylist(metrics_datas)

table.append(df)
print(df)


pyarrow.Table
serviceName: string
createdTime: timestamp[us]
metricData: string
language: string
----
serviceName: [["kafka-srv-5","kafka-srv-5","kafka-srv-5","kafka-srv-5","kafka-srv-5","kafka-srv-5","kafka-srv-5","kafka-srv-5","kafka-srv-5"]]
createdTime: [[2024-07-24 10:09:42.797115,2024-07-24 10:09:42.797115,2024-07-24 10:09:42.797115,2024-07-24 10:09:42.797115,2024-07-24 10:09:42.797115,2024-07-24 10:09:42.797115,2024-07-24 10:09:42.797115,2024-07-24 10:09:42.797115,2024-07-24 10:09:42.797115]]
metricData: [["{"resourceMetrics": [{"resource": {"attributes": [{"key": "container.id", "value": {"stringValue": "a0bf9fa1ba205cac19ae9584e73de46d3c8de0cbe0be787f6a216eead1c0003b"}}, {"key": "host.arch", "value": {"stringValue": "amd64"}}, {"key": "host.name", "value": {"stringValue": "kafka-srv-5-59bfc54888-2gkzx"}}, {"key": "k8s.container.name", "value": {"stringValue": "kafka-srv-5"}}, {"key": "k8s.deployment.name", "value": {"stringValue": "kafka-srv-5"}}, {"key": "k8s.namespace.name",